## 1. Install Packages

First, install necessary packages listed in `config.yml` for current notebook, including `express-py` which has `jarvis-tools` as dependency.

In [ ]:
from jupyterlite.utils import install_packages

await install_packages("import_material_from_jarvis_db_entry.ipynb")

## 2. Get Materials Data From JARVIS

Then, let's get the dataset containing 2D materials from JARVIS and wrap it into a pandas dataframe.


In [ ]:
import pandas as pd
from jarvis.db.figshare import data, get_jid_data

dft_2d = data("dft_2d")
df = pd.DataFrame(dft_2d)

## 3. Extract Structure and Convert to ESSE

Next, we extract an entry from the JARVIS dataset and convert it into ESSE format ready to be uploaded to Mat3ra.com.


In [ ]:
import json
from express import ExPrESS

## Note: JVASP-670 is an entry for MoTe2
jarvis_db_entry = json.dumps(json.loads(df[(df["jid"] == "JVASP-670")].to_json(orient="records"))[0])

kwargs = {
    "structure_string": jarvis_db_entry,
    "cell_type": "original",
    "structure_format": "jarvis-db-entry",
}

handler = ExPrESS("structure", **kwargs)
data = handler.property("material", **kwargs)

# To preview resulting JSON data
print(json.dumps(data, indent=4))

### 4. Pass data to the outside runtime

Finally, we can pass the material to the Materials Designer runtime to use it on the platform.

In [ ]:
from jupyterlite.utils import set_data

output_materials = [data]
set_data("materials", output_materials)